In [ ]:
# DATA LOADER

In [22]:
import torch
from torch.utils.data import Dataset
import torch.nn as nn

In [26]:
def create_squares_csv(lower_limit = 1, upper_limit=1000, filename = "Squares_1_1000.csv"):
    """Create a csv file with numbers and their squares"""
    
    with open(filename, 'w') as f:
        for num in range(lower_limit, upper_limit+1):
            print(str(num) + " " + str(num**2), file=f)       

In [27]:
create_squares_csv()

In [45]:
class SquaresDataset(Dataset):
    def __init__(self, filename="Squares_1_1000.csv"):
        super().__init__()              
        with open(filename, 'r') as f:
            self.samples = f.read().splitlines()
        
        for index, point in enumerate(self.samples):
            self.samples[index] = [int(num) for num in point.split()]
    
    def __len__(self):
        return len(self.samples)
    
    def __getitem__(self, idx):
        return self.samples[idx]

In [46]:
class FunctionApproximator(nn.Module):
    def __init__(self, layers=2, hidden_units= [6,4], non_linearity = "relu"):
        super().__init__()
        
        self.layer_list = nn.ModuleList()
        self.layer_list.append(nn.Linear(1, hidden_units[0]))
        
        for layer in range(layers-1):
            self.layer_list.append(nn.Linear(hidden_units[layer], hidden_units[layer+1]))
            
        self.layer_list.append(nn.Linear(hidden_units[-1],1))
        
        if non_linearity == 'relu':
            self.non_linearity = torch.relu
        
    def forward(self, x):
        h = x
        
        for index, layer in enumerate(self.layer_list[:-1]):
            h = layer(h)
            h = self.non_linearity(h)
        
        output =  self.layer_list[-1](h)
        return output

In [50]:
if __name__ == '__main__':
    import torch.utils.data as td
    from torch.optim import Adam
    
    RANDOM_SEED = 18

    if torch.cuda.is_available():
        device = 'cuda'
        torch.cuda.manual_seed(RANDOM_SEED)
        torch.set_default_tensor_type(torch.cuda.FloatTensor)
        kwargs = {'num_workers': 0, 'pin_memory': True}
    else:
        device = 'cpu'
        torch.manual_seed(RANDOM_SEED)
        torch.set_default_tensor_type(torch.FloatTensor)
        kwards = {}
        
    trainset = SquaresDataset(filename="Squares_1_1000.csv")
    testset = SquaresDataset(filename="Squares_1_1000.csv") #TODO Change to different numbers

    LR = 3e-4 # It's Magic! https://twitter.com/karpathy/status/801621764144971776?lang=en
    DATAPOINTS = len(trainset)
    BATCH_SIZE = 512
    BATCHES = DATAPOINTS / BATCH_SIZE

    net = FunctionApproximator(layers=2, hidden_units= [6,4], non_linearity = "relu")

    criterion = nn.MSELoss()
    net = net.to(device)

    train_loader = td.DataLoader(trainset, batch_size=BATCH_SIZE, shuffle=True, **kwargs)
    val_loader = td.DataLoader(testset, batch_size=BATCH_SIZE, shuffle=True, **kwargs)

    optimizer = Adam(net.parameters(), lr=LR)


In [52]:
trainset[4]

[5, 25]

In [51]:
trainset[100]

[101, 10201]

In [32]:
x,y

NameError: name 'x' is not defined

In [35]:
from tqdm import tqdm_notebook as tqdm
def train_model(start_steps = 0, end_steps = 5, net=None, model_name = "prelim", 
                train_loader = train_loader, val_loader = val_loader, 
                logger_level = 20):
    
    for epoch in tqdm(range(start_steps, end_steps)):
        count = epoch-start_steps+1
        net.train()
        
        #Epoch begins
        epoch_loss = 0.0
        for d in tqdm(train_loader):
            x, y = d.split()
            x, y = int(x), int(y)
            
            optimizer.zero_grad()
            x = x.to(device)
            with torch.no_grad():
                d = d.to(device)
            y = net(x)
            loss = torch.sqrt(criterion(y, d))   #RMSE Loss   
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
            
            #Break if NaN encountered
            if torch.isnan(loss) or torch.isinf(loss):
                logging.info(f"Loss value: {loss.item()}")
                logging.info("NaN/inf occured at:")
                logging.info(f"{x}\n")
                logging.info(f"{d}\n")
                logging.info(f"Original x was : {original_x}")
                NaN_flag = True
                break

            logging.debug(f"Count: {count}, Loss :{loss}")
            
        if NaN_flag: break   #Stop training if NaN encountered
        
        #Print to screen every few epochs    
        if count%LOG_INTERVAL == 0:
            print(f"Epoch number:{epoch} Loss: {epoch_loss:.4f}")  
            
        #Training artifacts
        if model_name not in os.listdir():
            os.makedirs(model_name+"/artifacts/saved_model/")

        #Write to loss file every epoch
        with open(model_name+"/artifacts/loss_curve",mode = 'a+') as f:
            f.write(f"Epoch_number: {epoch} Loss: {epoch_loss:.4f}\n")
            
        #Validation curve
        val_loss = 0.0
        net.eval()
        for x,d in val_loader:
            x[torch.isnan(x)]=0
            d[torch.isnan(d)]=0
            x = x.to(device)
            with torch.no_grad():
                d= d.to(device)
            y = net(x)
            loss = torch.sqrt(criterion(y,d))
            val_loss+=loss
        net.train()
        #Write Val loss to file every epoch
        with open(model_name+"/artifacts/val_loss_curve",mode = 'a+') as f:
            f.write(f"Epoch_number: {epoch} Loss: {val_loss:.4f}\n")
        
        #Save model every few epochs
        if epoch%SAVE_INTERVAL== 0:
            torch.save(net.state_dict(),f"./{model_name}/artifacts/saved_model/model_at_epoch{epoch}")
        #Epoch Ends